In [1]:
#Removing less than 20
import numpy as np
import glob
import SimpleITK as sitk
from tqdm import tqdm
import pydicom
from skimage import measure, color
import random

file = glob.glob('./data/*')
num=[]
image = []
mask = []
imagefile = []
maskfile = []

for i in file:
    mfile = glob.glob(i + '/*.nii.gz')
    if len(mfile) == 0:
        mfile = glob.glob(i + '/*.nii')
    maskfile.append(mfile[0])

for i in tqdm(range(len(file))):
    dicomp=glob.glob(file[i]+'/*.dcm')
    ds=sitk.ReadImage(dicomp[0])
    files = file[i].replace('data', 'brain_data')
    iii = sitk.ReadImage(files + '/image.nii.gz')
    #    iii = sitk.Cast(iii, sitk.sitkFloat32)
    #    img_mask = sitk.BinaryNot(sitk.BinaryThreshold(iii, 0, 0))
    #    iii = sitk.N4BiasFieldCorrection(iii, img_mask)
    ii = sitk.GetArrayFromImage(iii)
    ii=np.clip(ii,0,2000)
    m = sitk.ReadImage(maskfile[i])
    mm = sitk.GetArrayFromImage(m)
    unitVol=np.prod(ds.GetSpacing())
    labels = measure.label(mm)
    region = measure.regionprops(labels)
    for j in region:
        bbox = j.bbox
        (x, y, z) = ((bbox[4] + bbox[1]) / 2, (bbox[5] + bbox[2]) / 2,
                     (bbox[3] + bbox[0]) / 2)
        (x, y, z) = (int(x), int(y), int(z))
        counts=np.sum(mm[bbox[0]:bbox[-3],bbox[1]:bbox[-2],bbox[2]:bbox[-1]])
        if counts*unitVol>20:       
            if z - 40 <= 0:
                p = ii[0:80, x - 64:x + 64, y - 64:y + 64]
                pp = mm[0:80, x - 64:x + 64, y - 64:y + 64]
            elif z + 40 >= len(mm):
                p = ii[len(mm) - 80:len(mm), x - 64:x + 64, y - 64:y + 64]
                pp = mm[len(mm) - 80:len(mm), x - 64:x + 64, y - 64:y + 64]
            else:
                p = ii[z - 40:z + 40, x - 64:x + 64, y - 64:y + 64]
                pp = mm[z - 40:z + 40, x - 64:x + 64, y - 64:y + 64]

            if np.any(pp[:,0:30,:])==1 or np.any(pp[:,100:128,:])==1 or np.any(pp[:,:,0:30])==1 or np.any(pp[:,:,100:128])==1:
                xr=bbox[4]-x
                yr=bbox[5]-y
                xlim=(64-xr-15)
                ylim=(64-yr-15)
                xd=[i for i in range(x-xlim,x+xlim)]
                random.shuffle(xd)
                yd=[i for i in range(y-ylim,y+ylim)]
                random.shuffle(yd)

                for iy in xd:
                    for jy in yd:
                        if z - 40 <= 0:
                            p = ii[0:80, iy - 64:iy + 64, jy - 64:jy + 64]
                            pp = mm[0:80, iy - 64:iy + 64, jy - 64:jy + 64]
                        elif z + 40 >= len(mm):
                            p = ii[len(mm) - 80:len(mm), iy - 64:iy + 64, jy - 64:jy + 64]
                            pp = mm[len(mm) - 80:len(mm), iy - 64:iy + 64, jy - 64:jy + 64]
                        else:
                            p = ii[z - 40:z + 40, iy - 64:iy + 64, jy - 64:jy + 64]
                            pp = mm[z - 40:z + 40, iy - 64:iy + 64, jy - 64:jy + 64]

                        if np.any(pp[:,0:30,:])==1 or np.any(pp[:,100:128,:])==1 or np.any(pp[:,:,0:30])==1 or np.any(pp[:,:,100:128])==1:
                            pass
                        else:
                            break
                    else:
                        continue
                    break
                else:
                    break

            image.append(p)
            mask.append(pp)

iArr = np.zeros((len(image), 80, 128, 128))
mArr = np.zeros((len(image), 80, 128, 128))
for i in range(len(image)):
    mean = np.mean(image[i])
    std = np.std(image[i])
    iArr[i, :, :, :] = (image[i] - mean) / (std)
    mArr[i, :, :, :] = mask[i]
del image
del mask
            
from tqdm import tqdm
from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,    
    CenterCrop,    
    Crop,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomContrast,
    RandomGamma,
    RandomBrightness,
    ShiftScaleRotate,
    Flip
)

sf = np.zeros((len(iArr),80,128,128))
sfm = np.zeros((len(iArr),80,128,128))

print('*'*30+'水平翻转开始'+'*'*30)

for i in tqdm(range(len(iArr))):
    image=iArr[i]
    mask=mArr[i]
    aug = HorizontalFlip(p=0.9)
    augmented = aug(image=image, mask=mask)

    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    sf[i]=image_h_flipped
    sfm[i]=mask_h_flipped

cf = np.zeros((len(iArr),80,128,128))
cfm = np.zeros((len(iArr),80,128,128))

print('*'*30+'垂直翻转开始'+'*'*30)
for i in tqdm(range(len(iArr))):
    image=iArr[i]
    mask=mArr[i]
    aug =VerticalFlip(p=0.9)
    augmented = aug(image=image, mask=mask)

    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    cf[i]=image_h_flipped
    cfm[i]=mask_h_flipped

print('*'*30+'弹性变换开始'+'*'*30)


wg = np.zeros((len(iArr),80,128,128))
wgm = np.zeros((len(iArr),80,128,128))
for i in tqdm(range(len(iArr))):
    image=iArr[i]
    mask=mArr[i]
    aug = GridDistortion(p=1)
    augmented = aug(image=image, mask=mask)

    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    wg[i]=image_h_flipped
    wgm[i]=mask_h_flipped

 
r = np.zeros((len(iArr),80,128,128))
rm = np.zeros((len(iArr),80,128,128))
for i in tqdm(range(len(iArr))):
    image=iArr[i]
    mask=mArr[i]
    aug = ShiftScaleRotate(shift_limit=0.3, scale_limit=1e-9, rotate_limit=1e-9, p=1)
    augmented = aug(image=image, mask=mask)

    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    r[i]=image_h_flipped
    rm[i]=mask_h_flipped

rr = np.zeros((len(iArr),80,128,128))
rrm = np.zeros((len(iArr),80,128,128))
for i in tqdm(range(len(iArr))):
    image=iArr[i]
    mask=mArr[i]
    aug = ShiftScaleRotate(shift_limit=1e-9, scale_limit=(-0.2,0.2), rotate_limit=1e-9, p=1)
    augmented = aug(image=image, mask=mask)

    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    rr[i]=image_h_flipped
    rrm[i]=mask_h_flipped

rrr = np.zeros((len(iArr),80,128,128))
rrrm = np.zeros((len(iArr),80,128,128))
for i in tqdm(range(len(iArr))):
    image=iArr[i]
    mask=mArr[i]
    aug = ShiftScaleRotate(shift_limit=1e-9, scale_limit=1e-9, rotate_limit=(-45,45), p=1)
    augmented = aug(image=image, mask=mask)

    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    rrr[i]=image_h_flipped
    rrrm[i]=mask_h_flipped
    
img=np.concatenate((iArr,sf,cf,wg,r,rr,rrr),axis=0) 
msk=np.concatenate((mArr,sfm,cfm,wgm,rm,rrm,rrrm),axis=0) 

fimage=np.ndarray([len(msk),1,80,128,128],dtype=np.float32)
fmask=np.ndarray([len(msk),1,80,128,128],dtype=np.float32)
for i in tqdm(range(len(msk))):
#    min=np.mean(img[i])
#    max=np.std(img[i])
    fimage[i]=img[i]
    fmask[i]=msk[i]

rand_i = np.random.choice(range(len(fmask)),size=len(fmask),replace=False)
finali=fimage[rand_i]
finalm=fmask[rand_i]
np.save('./3Dran.npy',finali)
np.save('./3Dranm.npy',finalm)

ModuleNotFoundError: No module named 'SimpleITK'

In [1]:
#Removing less than 20 and 3D clahe
import numpy as np
import mclahe as mc
import glob
import SimpleITK as sitk
from tqdm import tqdm
import pydicom
from skimage import measure, color
import random

file = glob.glob('./data/*')
num=[]
image = []
mask = []
imagefile = []
maskfile = []

for i in file:
    mfile = glob.glob(i + '/*.nii.gz')
    if len(mfile) == 0:
        mfile = glob.glob(i + '/*.nii')
    maskfile.append(mfile[0])

for i in tqdm(range(len(file))):
    dicomp=glob.glob(file[i]+'/*.dcm')
    ds=sitk.ReadImage(dicomp[0])
    files = file[i].replace('data', 'brain_data')
    iii = sitk.ReadImage(files + '/image.nii.gz')
    #    iii = sitk.Cast(iii, sitk.sitkFloat32)
    #    img_mask = sitk.BinaryNot(sitk.BinaryThreshold(iii, 0, 0))
    #    iii = sitk.N4BiasFieldCorrection(iii, img_mask)
    ii = sitk.GetArrayFromImage(iii)
    ii=np.clip(ii,0,2000)
    m = sitk.ReadImage(maskfile[i])
    mm = sitk.GetArrayFromImage(m)
    unitVol=np.prod(ds.GetSpacing())
    labels = measure.label(mm)
    region = measure.regionprops(labels)
    for j in region:
        bbox = j.bbox
        (x, y, z) = ((bbox[4] + bbox[1]) / 2, (bbox[5] + bbox[2]) / 2,
                     (bbox[3] + bbox[0]) / 2)
        (x, y, z) = (int(x), int(y), int(z))
        counts=np.sum(mm[bbox[0]:bbox[-3],bbox[1]:bbox[-2],bbox[2]:bbox[-1]])
        if counts*unitVol>20:       
            if z - 40 <= 0:
                p = ii[0:80, x - 64:x + 64, y - 64:y + 64]
                pp = mm[0:80, x - 64:x + 64, y - 64:y + 64]
            elif z + 40 >= len(mm):
                p = ii[len(mm) - 80:len(mm), x - 64:x + 64, y - 64:y + 64]
                pp = mm[len(mm) - 80:len(mm), x - 64:x + 64, y - 64:y + 64]
            else:
                p = ii[z - 40:z + 40, x - 64:x + 64, y - 64:y + 64]
                pp = mm[z - 40:z + 40, x - 64:x + 64, y - 64:y + 64]

            if np.any(pp[:,0:30,:])==1 or np.any(pp[:,100:128,:])==1 or np.any(pp[:,:,0:30])==1 or np.any(pp[:,:,100:128])==1:
                xr=bbox[4]-x
                yr=bbox[5]-y
                xlim=(64-xr-15)
                ylim=(64-yr-15)
                xd=[i for i in range(x-xlim,x+xlim)]
                random.shuffle(xd)
                yd=[i for i in range(y-ylim,y+ylim)]
                random.shuffle(yd)

                for iy in xd:
                    for jy in yd:
                        if z - 40 <= 0:
                            p = ii[0:80, iy - 64:iy + 64, jy - 64:jy + 64]
                            pp = mm[0:80, iy - 64:iy + 64, jy - 64:jy + 64]
                        elif z + 40 >= len(mm):
                            p = ii[len(mm) - 80:len(mm), iy - 64:iy + 64, jy - 64:jy + 64]
                            pp = mm[len(mm) - 80:len(mm), iy - 64:iy + 64, jy - 64:jy + 64]
                        else:
                            p = ii[z - 40:z + 40, iy - 64:iy + 64, jy - 64:jy + 64]
                            pp = mm[z - 40:z + 40, iy - 64:iy + 64, jy - 64:jy + 64]

                        if np.any(pp[:,0:30,:])==1 or np.any(pp[:,100:128,:])==1 or np.any(pp[:,:,0:30])==1 or np.any(pp[:,:,100:128])==1:
                            pass
                        else:
                            break
                    else:
                        continue
                    break
                else:
                    break
            voxel=mc.mclahe(p,kernel_size=[16,84,84],n_bins=256,clip_limit=0.01)
            image.append(voxel)
            mask.append(pp)

iArr = np.zeros((len(image), 80, 128, 128))
mArr = np.zeros((len(image), 80, 128, 128))
for i in range(len(image)):
    mean = np.mean(image[i])
    std = np.std(image[i])
    iArr[i, :, :, :] = (image[i] - mean) / (std)
    mArr[i, :, :, :] = mask[i]
del image
del mask
            
from tqdm import tqdm
from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,    
    CenterCrop,    
    Crop,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomContrast,
    RandomGamma,
    RandomBrightness,
    ShiftScaleRotate,
    Flip
)

sf = np.zeros((len(iArr),80,128,128))
sfm = np.zeros((len(iArr),80,128,128))

print('*'*30+'水平翻转开始'+'*'*30)

for i in tqdm(range(len(iArr))):
    image=iArr[i]
    mask=mArr[i]
    aug = HorizontalFlip(p=0.9)
    augmented = aug(image=image, mask=mask)

    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    sf[i]=image_h_flipped
    sfm[i]=mask_h_flipped

cf = np.zeros((len(iArr),80,128,128))
cfm = np.zeros((len(iArr),80,128,128))

print('*'*30+'垂直翻转开始'+'*'*30)
for i in tqdm(range(len(iArr))):
    image=iArr[i]
    mask=mArr[i]
    aug =VerticalFlip(p=0.9)
    augmented = aug(image=image, mask=mask)

    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    cf[i]=image_h_flipped
    cfm[i]=mask_h_flipped

print('*'*30+'弹性变换开始'+'*'*30)


wg = np.zeros((len(iArr),80,128,128))
wgm = np.zeros((len(iArr),80,128,128))
for i in tqdm(range(len(iArr))):
    image=iArr[i]
    mask=mArr[i]
    aug = GridDistortion(p=1)
    augmented = aug(image=image, mask=mask)

    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    wg[i]=image_h_flipped
    wgm[i]=mask_h_flipped

 
r = np.zeros((len(iArr),80,128,128))
rm = np.zeros((len(iArr),80,128,128))
for i in tqdm(range(len(iArr))):
    image=iArr[i]
    mask=mArr[i]
    aug = ShiftScaleRotate(shift_limit=0.3, scale_limit=1e-9, rotate_limit=1e-9, p=1)
    augmented = aug(image=image, mask=mask)

    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    r[i]=image_h_flipped
    rm[i]=mask_h_flipped

rr = np.zeros((len(iArr),80,128,128))
rrm = np.zeros((len(iArr),80,128,128))
for i in tqdm(range(len(iArr))):
    image=iArr[i]
    mask=mArr[i]
    aug = ShiftScaleRotate(shift_limit=1e-9, scale_limit=(-0.2,0.2), rotate_limit=1e-9, p=1)
    augmented = aug(image=image, mask=mask)

    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    rr[i]=image_h_flipped
    rrm[i]=mask_h_flipped

rrr = np.zeros((len(iArr),80,128,128))
rrrm = np.zeros((len(iArr),80,128,128))
for i in tqdm(range(len(iArr))):
    image=iArr[i]
    mask=mArr[i]
    aug = ShiftScaleRotate(shift_limit=1e-9, scale_limit=1e-9, rotate_limit=(-45,45), p=1)
    augmented = aug(image=image, mask=mask)

    image_h_flipped = augmented['image']
    mask_h_flipped = augmented['mask']
    rrr[i]=image_h_flipped
    rrrm[i]=mask_h_flipped
    
img=np.concatenate((iArr,sf,cf,wg,r,rr,rrr),axis=0) 
msk=np.concatenate((mArr,sfm,cfm,wgm,rm,rrm,rrrm),axis=0) 

fimage=np.ndarray([len(msk),1,80,128,128],dtype=np.float32)
fmask=np.ndarray([len(msk),1,80,128,128],dtype=np.float32)
for i in tqdm(range(len(msk))):
#    min=np.mean(img[i])
#    max=np.std(img[i])
    fimage[i]=img[i]
    fmask[i]=msk[i]

rand_i = np.random.choice(range(len(fmask)),size=len(fmask),replace=False)
finali=fimage[rand_i]
finalm=fmask[rand_i]
np.save('./3Dclahe.npy',finali)
np.save('./3Dclahem.npy',finalm)

100%|██████████| 105/105 [06:56<00:00,  3.97s/it]


******************************水平翻转开始******************************


100%|██████████| 120/120 [00:00<00:00, 134.31it/s]


******************************垂直翻转开始******************************


100%|██████████| 120/120 [00:00<00:00, 156.83it/s]


******************************弹性变换开始******************************


100%|██████████| 840/840 [00:01<00:00, 421.29it/s]
